In [1]:
!pip install yfinance
import yfinance as yf
import numpy as np
import pandas as pd
from datetime import datetime
import calendar

def calculate_monthly_rolling_sharpe_ratios(data, ticker='SPY', start_date='2023-11-30', end_date='2024-11-30', lookback_months=12):
    """
    Calculate rolling Sharpe ratios for each month-end from start_date to end_date.

    Parameters:
    - data: DataFrame containing price data for the ticker
    - ticker: The ticker symbol to analyze (default: 'SPY')
    - start_date: The starting date for analysis
    - end_date: The ending date for analysis
    - lookback_months: Number of months to use for each Sharpe ratio calculation

    Returns:
    - DataFrame with dates and corresponding Sharpe ratios
    """
    # Convert dates to datetime if they're strings
    if isinstance(start_date, str):
        start_date = pd.to_datetime(start_date)
    if isinstance(end_date, str):
        end_date = pd.to_datetime(end_date)

    # Extend the data range to include enough history for the first calculation
    extended_start = pd.to_datetime(start_date) - pd.DateOffset(months=lookback_months)

    # Filter data for the ticker within the extended date range
    ticker_data = data[ticker][extended_start:end_date]

    # Calculate monthly returns
    monthly_returns = ticker_data.resample('M').last().pct_change().dropna()

    # Generate a list of month-end dates from start_date to end_date
    current_date = pd.to_datetime(start_date)
    end_date = pd.to_datetime(end_date)

    results = []

    while current_date <= end_date:
        # Find the last day of the current month
        last_day = calendar.monthrange(current_date.year, current_date.month)[1]
        month_end = pd.Timestamp(datetime(current_date.year, current_date.month, last_day))

        # Get data up to this month-end
        history_end = month_end
        history_start = history_end - pd.DateOffset(months=lookback_months)

        # Get the returns in this window
        window_returns = monthly_returns[(monthly_returns.index >= history_start) &
                                        (monthly_returns.index <= history_end)]

        # Only calculate if we have enough data
        if len(window_returns) >= lookback_months - 1:  # Need at least lookback_months-1 return values
            # Calculate annualized return and standard deviation
            annualized_return = (1 + window_returns.mean()) ** 12 - 1
            annualized_std = window_returns.std() * np.sqrt(12)

            # Calculate Sharpe ratio (assuming 0 risk-free rate for simplicity)
            sharpe_ratio = annualized_return / annualized_std

            results.append({
                'date': month_end,
                'sharpe_ratio': sharpe_ratio
            })

        # Move to the next month
        current_date = month_end + pd.DateOffset(days=1)

    # Convert results to DataFrame
    results_df = pd.DataFrame(results)

    return results_df


import pandas as pd
import yfinance as yf  # A popular library for downloading financial data

# Download data
start_date = '2022-11-30'  # Extended to have enough history
end_date = '2024-11-30'
tickers = ['SPY']  # Can include multiple tickers

# Download data
data = yf.download(tickers, start=start_date, end=end_date)['Close']

# If you have only one ticker, convert to DataFrame with ticker as column name
if isinstance(data, pd.Series):
    data = data.to_frame(name=tickers[0])

# Now you can use the function
sharpe_ratios = calculate_monthly_rolling_sharpe_ratios(
    data,
    ticker='SPY',
    start_date='2023-11-30',
    end_date='2024-11-30'
)
sharpe_ratios

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed
<ipython-input-1-6aa8cce87118>:35: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthly_returns = ticker_data.resample('M').last().pct_change().dropna()


,date,sharpe_ratio
0,2023-11-30,0.922380
1,2023-12-31,1.168919
2,2024-01-31,1.902685
3,2024-02-29,1.854181
4,2024-03-31,2.477436
5,2024-04-30,1.635504
6,2024-05-31,1.865492
7,2024-06-30,2.114160
8,2024-07-31,1.777293
9,2024-08-31,1.709807
